ExE Interaction Analysis
------------------------


<h4>Part 1: Queries</h4>

CREATE A SINGLE SEARCH PROCESS
<ol>
<li>Entry of a List of IDs and Descriptions</li>
<li>Rotate word_to_term for the Descriptions</li>
<li>Search the TermMap</li>
<li>Super table (ID, Description, Word, Term, Term_1, Term_2, ID_1, ID_2,
    Groups/Categories, counts)</li>
</ol>
With this super table, the user can perform the cleaning as needed, thus
eliminating categories that are not of interest

<h4>Part 2: ExE LRT ANALYSY</h4>
<ol>
<li>Define Main Table, Outcomes and Covariantes</li>
<li>Run the ExE Interation by Outcome and Map Pairs</li>
</ol>

<lr>

<h3>Set Environment</h3>


In [1]:
import os
import sys
import pandas as pd
from pathlib import Path
# Set Local Environment to consider igem local folder as a Package
current_path = Path.cwd()
# v_root = Path.cwd().parents[0]
# sys.path.append(os.path.abspath(v_root))
# v_path = v_root / "ExE_InterationAnalysis" / "files"
v_path = current_path / "files"



In [2]:
# Import IGEM folder as a Package
# Return rpy2 msm.
from igem import epc, ge

rpy2 ModuleSpec(name='rpy2', loader=<_frozen_importlib_external.SourceFileLoader object at 0x13d1b9d90>, origin='/Users/andrerico/DEV/Tests/igem/igem_Notebook/venv/lib/python3.9/site-packages/rpy2/__init__.py', submodule_search_locations=['/Users/andrerico/DEV/Tests/igem/igem_Notebook/venv/lib/python3.9/site-packages/rpy2'])


Set django async connection

In [3]:
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'src.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

Step 1: Get IGEM Terms from a ID and Description NHAMES List

In [4]:
df_nhames_desc = epc.load.from_csv(str(v_path / "nhanes_description.csv"))

Loaded 3,174 observations of 36 variables


In [5]:
df_nhames_desc = epc.modify.colfilter(df_nhames_desc, only=["var", "var_desc"])
df_nhames_desc["var_desc"].to_csv(
    str(v_path / "nhanes_description_igem.csv"), index=False
)

Running colfilter
--------------------------------------------------------------------------------
Keeping 2 of 36 variables:
	0 of 0 binary variables
	0 of 0 categorical variables
	0 of 26 continuous variables
	2 of 10 unknown variables


In [6]:
# From NHAMES get IGEM Terms
# df_ge_terms = ge.filter.word_to_term(str(v_path /
#   "nhanes_description_strings.csv"))
df_terms = epc.load.from_csv(
    str(v_path / "nhanes_description_terms.csv"), index_col=False
)
df_terms = epc.modify.colfilter(
    df_terms, only=["fatores", "term"]
)  # TODO: change output format

Loaded 3,304 observations of 6 variables
Running colfilter
--------------------------------------------------------------------------------
Keeping 2 of 6 variables:
	0 of 0 binary variables
	0 of 0 categorical variables
	0 of 1 continuous variables
	2 of 5 unknown variables


Step 2: Get TermMap from Terms founds in step 1

In [7]:
# Delete Terms that is not target study
for x in ["anat", "go", "path", "meta:hmdb0002111"]:
    df_terms = df_terms[~df_terms["term"].astype(str).str.startswith(x)]

In [8]:
df_terms = df_terms.dropna()

In [9]:
df_terms = df_terms.drop_duplicates(subset="fatores", keep="last")

In [10]:
# Return NHANES ID to df_terms
df_terms["fatores"] = df_terms["fatores"].str.lower()
df_nhames_desc["var_desc"] = df_nhames_desc["var_desc"].str.lower()
df_terms = df_terms.merge(
    df_nhames_desc, left_on="fatores", right_on="var_desc", how="left"
)

In [11]:
# Convert DF column term to Terms List
list_term = df_terms["term"].tolist()

In [12]:
# Get all Term Map from Terms List
df_term_map = ge.filter.term_map(term=list_term)

In [13]:
# Clear fields from Term Map DF
df_term_map = epc.modify.colfilter(df_term_map, only=["term_1", "term_2"])

Running colfilter
--------------------------------------------------------------------------------
Keeping 2 of 13 variables:
	0 of 0 binary variables
	0 of 0 categorical variables
	0 of 3 continuous variables
	2 of 10 unknown variables


Step 3: Replace Terms ID by NHANES ID

In [14]:
# Replace Terms by NHANES ID on df_term_map
# TODO: I could use epc.modify.merge_variables because we don`t have to key
df_nhanes_map = df_term_map.merge(
    df_terms, left_on="term_1", right_on="term", how="left"
)

In [15]:
df_nhanes_map = df_nhanes_map.merge(
    df_terms, left_on="term_2", right_on="term", how="left"
)

In [16]:
df_nhanes_map = epc.modify.colfilter(
    df_nhanes_map, only=["var_x", "var_desc_x", "var_y", "var_desc_y"]
)

Running colfilter
--------------------------------------------------------------------------------
Keeping 4 of 10 variables:
	0 of 0 binary variables
	0 of 0 categorical variables
	0 of 0 continuous variables
	4 of 10 unknown variables


STEP 4: Clean df_nhanes_map to match the study target

In [17]:
# Delete maps without one of the nhanes ID
df_nhanes_map = df_nhanes_map.dropna()

In [18]:
# Define list of nhanes ID that will not match the study target
list_remove = [
    "pneu",
    "current_asthma",
    "ever",
    "any",
    "ATORVASTATIN",
    "AZITHROMYCIN",
    "CARVEDILOL",
    "hepb",
    "FENOFIBRATE",
    "FLUOXETINE",
    "BUPROPION",
    "GLYBURIDE",
    "ASPIRIN",
    "heroin",
    "ALENDRONATE",
    "METFORMIN",
    "ESTRADIOL",
    "OMEPRAZOLE",
    "NIFEDIPINE",
    "PREDNISONE",
    "PIOGLITAZONE",
    "ROFECOXIB",
    "ALBUTEROL",
    "SPIRONOLACTONE",
    "SIMVASTATIN",
    "SERTRALINE",
    "LOVASTATIN",
    "LOSARTAN",
    "cocaine",
    "DIGOXIN",
    "CELECOXIB",
]

In [19]:
for i in list_remove:
    df_nhanes_map = df_nhanes_map[~df_nhanes_map["var_x"].str.contains(i)]
    df_nhanes_map = df_nhanes_map[~df_nhanes_map["var_y"].str.contains(i)]

# fix the df index
df_nhanes_map = df_nhanes_map.reset_index()

STEP 5: Define Main Table, Outcomes and Covariantes

In [20]:
# DataFrame to collect results
df_results_discover_final = pd.DataFrame()
df_results_replicate_final = pd.DataFrame()
list_results_discover = []
list_results_replicate = []

In [21]:
# Read NHANES Main Table
df_maintable = epc.load.from_csv(str(v_path / "MainTable.csv"))

Loaded 41,474 observations of 1,190 variables


In [22]:
# list of Outcomes
list_outcome = [
    "LBXHGB",
]

In [23]:
# list of Covariants
list_covariant = [
    "female",
    "black",
    "mexican",
    "other_hispanic",
    "other_eth",
    "SDDSRVYR",
    "BMXBMI",
    "SES_LEVEL",
    "RIDAGEYR",
    "LBXCOT",
    "IRON_mg",
]

STEP 6: Run the ExE Interation by Outcome and Map Pairs

In [24]:
df_nhanes_map = epc.load.from_csv(str(v_path / "gepairs.csv"), index_col=None)

Loaded 769 observations of 2 variables


In [25]:
df_nhanes_map = df_nhanes_map.rename(
    columns={"NHANESID_var1": "var_x", "NHANESID_var2": "var_y"}
)

In [29]:
list_remove = [
    "pneu",
    "current_asthma",
    "ever",
    "any",
    "ATORVASTATIN",
    "AZITHROMYCIN",
    "CARVEDILOL",
    "hepb",
    "FENOFIBRATE",
    "FLUOXETINE",
    "BUPROPION",
    "GLYBURIDE",
    "ASPIRIN",
    "heroin",
    "ALENDRONATE",
    "METFORMIN",
    "ESTRADIOL",
    "OMEPRAZOLE",
    "NIFEDIPINE",
    "PREDNISONE",
    "PIOGLITAZONE",
    "ROFECOXIB",
    "ALBUTEROL",
    "SPIRONOLACTONE",
    "SIMVASTATIN",
    "SERTRALINE",
    "LOVASTATIN",
    "LOSARTAN",
    "cocaine",
    "DIGOXIN",
    "CELECOXIB",
]

In [30]:
for i in list_remove:
    df_nhanes_map = df_nhanes_map[~df_nhanes_map["var_x"].str.contains(i)]
    df_nhanes_map = df_nhanes_map[~df_nhanes_map["var_y"].str.contains(i)]

In [31]:
# fix the df index
df_nhanes_map = df_nhanes_map.reset_index()
df_nhanes_map = epc.modify.colfilter(df_nhanes_map, skip="ID")

Running colfilter
--------------------------------------------------------------------------------
Keeping 2 of 3 variables:
	0 of 0 binary variables
	0 of 0 categorical variables
	0 of 1 continuous variables
	2 of 2 unknown variables


In [76]:
# FULL CODE
# Run each Outcome in defined list
for i_outcome in list_outcome:
    # Run each pair map in df_nhanes_map
    for i_mappair in df_nhanes_map.index:
        # get Exposomes
        e1 = df_nhanes_map["var_x"][i_mappair]
        e2 = df_nhanes_map["var_y"][i_mappair]

        # Keep only the necessary columns to run Interations Study
        df_maintable_exe = df_maintable.loc[
            :, list_covariant + list_outcome + list([e1, e2])
        ]
        df_maintable_exe = df_maintable_exe.fillna(0)

        # Filter data table to Discovery Data
        df_maintable_exe = df_maintable_exe[
            df_maintable_exe["SDDSRVYR"].isin([1, 2])
            ]

        # Run Interation Study
        Interation_Study = epc.analyze.interaction_study(
            data=df_maintable_exe,
            outcomes=i_outcome,
            interactions=[(e1, e2)],
            covariates=list_covariant,
        )

        # Save results in list: outcome/e1/e2/converged/LRT_pvalue/Bonfp
        list_results_discover.append(
            [
                Interation_Study.LRT_pvalue.index.levels[2][0],
                Interation_Study.LRT_pvalue.index.levels[0][0],
                Interation_Study.LRT_pvalue.index.levels[1][0],
                Interation_Study.Converged.values[0],
                Interation_Study.LRT_pvalue.values[0],
                Interation_Study.LRT_pvalue.values[0] * len(df_nhanes_map),
            ]
        )

    # Transf result list in df with Bonfp < 0.05
    df_results_discover = pd.DataFrame(
        list_results_discover,
        columns=[
            "Outcome", "Term1", "Term2", "Converged", "LRT_pvalue", "Bonfp"
            ],
    )
    df_results_discover = df_results_discover.loc[
        df_results_discover["Bonfp"] <= 0.05
        ]

    # Run Replicate for all discovery result with Bonfp < 0.05
    for x in df_results_discover.itertuples(index=False):
        e1 = x.Term1
        e2 = x.Term2

        # Keep only the necessary columns to run Interations Study
        df_maintable_exe = df_maintable.loc[
            :, list_covariant + list_outcome + list([e1, e2])
        ]
        df_maintable_exe = df_maintable_exe.fillna(0)

        # Filter data table to Replicate Data
        df_maintable_exe = df_maintable_exe[
            df_maintable_exe["SDDSRVYR"].isin([3, 4])
            ]

        # Run Interation Study
        Interation_Study = epc.analyze.interaction_study(
            data=df_maintable_exe,
            outcomes=i_outcome,
            interactions=[(e1, e2)],
            covariates=list_covariant,
        )

        # Save results in list: outcome/e1/e2/converged/LRT_pvalue/Bonfp
        list_results_replicate.append(
            [
                Interation_Study.LRT_pvalue.index.levels[2][0],
                Interation_Study.LRT_pvalue.index.levels[0][0],
                Interation_Study.LRT_pvalue.index.levels[1][0],
                Interation_Study.Converged.values[0],
                Interation_Study.LRT_pvalue.values[0],
                Interation_Study.LRT_pvalue.values[0] * len(
                    df_results_discover
                ),
            ]
        )

    # Transf result list in df with Bonfp < 0.05
    df_results_replicate = pd.DataFrame(
        list_results_replicate,
        columns=[
            "Outcome", "Term1", "Term2", "Converged", "LRT_pvalue", "Bonfp"
            ],
    )
    df_results_replicate = df_results_replicate.loc[
        df_results_replicate["Bonfp"] <= 0.05
    ]

    df_results_discover_final = df_results_discover_final.append(
        df_results_discover
    ).reset_index(drop=True)
    df_results_replicate_final = df_results_replicate_final.append(
        df_results_replicate
    ).reset_index(drop=True)

InteractionRegression
-------------------------
Continuous Outcome (family = Gaussian): 'LBXHGB'
Using 21,004 of 21,004 observations
Regressing 2 variables
	0 binary variables
	0 categorical variables
	2 continuous variables
	0 genotypes variables
Processing 1 interactions
-------------------------
Running 1 interactions using 12 processes...
	Finished Running 1 interactions
0 tests had an error
Completed Interaction Study for LBXHGB

Completed association study
InteractionRegression
-------------------------
Continuous Outcome (family = Gaussian): 'LBXHGB'
Using 21,004 of 21,004 observations
Regressing 2 variables
	0 binary variables
	0 categorical variables
	2 continuous variables
	0 genotypes variables
Processing 1 interactions
-------------------------
Running 1 interactions using 12 processes...
	Finished Running 1 interactions
0 tests had an error
Completed Interaction Study for LBXHGB

Completed association study
InteractionRegression
-------------------------
Continuous Outcome

/var/folders/tv/qf2nl0h10kn_55zkv8p1wy3r0000gn/T/ipykernel_52545/2611025735.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results_discover_final = df_results_discover_final.append(
/var/folders/tv/qf2nl0h10kn_55zkv8p1wy3r0000gn/T/ipykernel_52545/2611025735.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results_replicate_final = df_results_replicate_final.append(


In [77]:
print(df_results_discover_final)
print(df_results_replicate_final)

   Outcome                 Term1    Term2  Converged     LRT_pvalue  \
0   LBXHGB                LBXHCB   LBXBHC       True   5.963622e-06   
1   LBXHGB                LBXGLU   LBXHBC       True   8.608529e-15   
2   LBXHGB                LBXSGL   LBXHBC       True   1.637474e-42   
3   LBXHGB              DR1TNIAC   LBXHBC       True   9.060379e-09   
4   LBXHGB                LBXALD   LBXHBC       True   6.861305e-05   
5   LBXHGB                LBXODT   LBXHBC       True   3.197605e-07   
6   LBXHGB                LBXVIE   LBXHBC       True   5.429730e-53   
7   LBXHGB                LBXVID   LBXHBC       True   6.583205e-09   
8   LBXHGB               LBXPFOS   LBXHBS       True   1.580342e-13   
9   LBXHGB                LBXSGL   LBXHBS       True   0.000000e+00   
10  LBXHGB                LBXVID   LBXHBS       True  1.424215e-116   
11  LBXHGB                LBXODT   LBXHBS       True   1.035803e-32   
12  LBXHGB                LBXVIE   LBXHBS       True  3.347839e-196   
13  LB

<br>

-----------------------------------------------------------------------------------------------------------------

<h1 class={color:red}> DEBUG </h1>
Run one case only

In [78]:
# define variables
i_outcome = "LBXHGB"
# e1 = "LBXGLU"
# e2 = "LBXHBC"
e1 = "DR1TSFAT"
e2 = "DRDSDT1"

# Keep only the necessary columns to run Interations Study
df_maintable_exe = df_maintable.loc[
    :, list_covariant + list_outcome + list([e1, e2])
]
df_maintable_exe = df_maintable_exe.fillna(0)

In [79]:
 # Filter data table to Discovery Data
df_maintable_exe = df_maintable_exe[
    df_maintable_exe["SDDSRVYR"].isin([1, 2])
    ]

In [80]:
# Run Interation Study
Interation_Study = epc.analyze.interaction_study(
    data=df_maintable_exe,
    outcomes=i_outcome,
    interactions=[(e1, e2)],
    covariates=list_covariant,
)

InteractionRegression
-------------------------
Continuous Outcome (family = Gaussian): 'LBXHGB'
Using 21,004 of 21,004 observations
Regressing 2 variables
	0 binary variables
	0 categorical variables
	2 continuous variables
	0 genotypes variables
Processing 1 interactions
-------------------------
Running 1 interactions using 12 processes...
	Finished Running 1 interactions
0 tests had an error
Completed Interaction Study for LBXHGB

Completed association study


<br>

<h1> Run manually</h1>

Run regretions manually

In [39]:
import pandas as pd
import scipy
import statsmodels.api as sm
# from sklearn.linear_model import LinearRegression

In [40]:
## HEMOGLOBIN
# define variables
i_outcome = "LBXHGB"
# e1 = "LBXGLU"
# e2 = "LBXHBC"
e1 = "DR1TSFAT"
e2 = "DRDSDT1"

resultstable_dis = pd.DataFrame()
resultstable_rep = pd.DataFrame()

nested_table = df_maintable.loc[
    :,
    [
        "LBXHGB",
        "female",
        "black",
        "mexican",
        "other_hispanic",
        "other_eth",
        "SDDSRVYR",
        "BMXBMI",
        "SES_LEVEL",
        "RIDAGEYR",
        "LBXCOT",
        "IRON_mg",
    ],
]


In [41]:
# Keep only the necessary columns to run Interations Study
nested_table = df_maintable.loc[
    :, list_covariant + list_outcome + list([e1, e2])
]

In [42]:
nested_table = nested_table.fillna(0)

In [43]:
nested_table_dis = nested_table[nested_table["SDDSRVYR"].isin([1, 2])]

In [44]:
complex_table = nested_table

In [45]:
complex_table["interaction"] = complex_table[e1] * complex_table[e2]

In [46]:
complex_table_dis = complex_table[complex_table["SDDSRVYR"].isin([1, 2])]

In [65]:
# Regression
y1 = nested_table_dis["LBXHGB"]
X1 = nested_table_dis[
    [
        "female",
        "black",
        "mexican",
        "other_hispanic",
        "other_eth",
        "SDDSRVYR",
        "BMXBMI",
        "SES_LEVEL",
        "RIDAGEYR",
        "LBXCOT",
        "IRON_mg",
        e1,
        e2,
    ]
]

In [66]:
X1 = sm.add_constant(X1)
nested = sm.OLS(y1, X1).fit()
nested_ll = nested.llf
print(nested_ll)

-60660.26874769988


In [68]:
y2 = complex_table_dis["LBXHGB"]
X2 = complex_table_dis[
    [
        "female",
        "black",
        "mexican",
        "other_hispanic",
        "other_eth",
        "SDDSRVYR",
        "BMXBMI",
        "SES_LEVEL",
        "RIDAGEYR",
        "LBXCOT",
        "IRON_mg",
        e1,
        e2,
        "interaction",
    ]
]

In [64]:
X2 = sm.add_constant(X2)
complex = sm.OLS(y2, X2).fit()
complex_ll = complex.llf
print(complex_ll)

-60660.26874769988


In [70]:
# STEP 3: Perform the Log-Likelihood Test
# Next, we’ll use the following code to perform the log-likelihood test:
# calculate likelihood ratio Chi-Squared test statistic
LR_statistic = -2 * (nested_ll - complex_ll)
print(LR_statistic)

-0.0


In [72]:
# process to find degrees of freedom
f = nested.df_resid - complex.df_resid
f

0.0

In [71]:
# calculate p-value of test statistic using 1 degrees of freedom
p_val = scipy.stats.chi2.sf(LR_statistic, 1)
p_val_f = scipy.stats.chi2.sf(LR_statistic, f)
print(p_val)

1.0
